In this Notebook, we run a baseline with
- Raw Unet Structure
- BCELoss
with full training - validation - submission code.

## Define Unet

In [8]:
import torch
import torch.nn as nn

# ================= Unet ========================
# Ref: https://www.kaggle.com/code/balraj98/unet-for-building-segmentation-pytorch
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.double_conv(x)

class DownBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DownBlock, self).__init__()
        self.double_conv = DoubleConv(in_channels, out_channels)
        self.down_sample = nn.MaxPool2d(2)

    def forward(self, x):
        skip_out = self.double_conv(x)
        down_out = self.down_sample(skip_out)
        return (down_out, skip_out)

class UpBlock(nn.Module):
    def __init__(self, in_channels, out_channels, up_sample_mode):
        super(UpBlock, self).__init__()
        if up_sample_mode == 'conv_transpose':
            self.up_sample = nn.ConvTranspose2d(in_channels-out_channels, in_channels-out_channels, kernel_size=2, stride=2)
        elif up_sample_mode == 'bilinear':
            self.up_sample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            raise ValueError("Unsupported `up_sample_mode` (can take one of `conv_transpose` or `bilinear`)")
        self.double_conv = DoubleConv(in_channels, out_channels)

    def forward(self, down_input, skip_input):
        x = self.up_sample(down_input)
        x = torch.cat([x, skip_input], dim=1)
        return self.double_conv(x)

class UNet(nn.Module):
    def __init__(self, in_ch, out_classes=1, up_sample_mode='conv_transpose'):
        super(UNet, self).__init__()
        self.up_sample_mode = up_sample_mode
        # Downsampling Path
        self.down_conv1 = DownBlock(in_ch, 64)
        self.down_conv2 = DownBlock(64, 128)
        self.down_conv3 = DownBlock(128, 256)
        self.down_conv4 = DownBlock(256, 512)

        # Bottleneck
        self.double_conv = DoubleConv(512, 1024)
        # Upsampling Path
        self.up_conv4 = UpBlock(512 + 1024, 512, self.up_sample_mode)
        self.up_conv3 = UpBlock(256 + 512, 256, self.up_sample_mode)
        self.up_conv2 = UpBlock(128 + 256, 128, self.up_sample_mode)
        self.up_conv1 = UpBlock(128 + 64, 64, self.up_sample_mode)

        # Final Convolution
        self.conv_last = nn.Conv2d(64, out_classes, kernel_size=1)

    def forward(self, x):
        x, skip1_out = self.down_conv1(x)
        x, skip2_out = self.down_conv2(x)
        x, skip3_out = self.down_conv3(x)
        x, skip4_out = self.down_conv4(x)
        x = self.double_conv(x)
        x = self.up_conv4(x, skip4_out)
        x = self.up_conv3(x, skip3_out)
        x = self.up_conv2(x, skip2_out)
        x = self.up_conv1(x, skip1_out)
        x = self.conv_last(x)
        return x

## Define Dataset
Here the RandomPatchLocDataset will draw random patch from the volume.

**Full visualize** can ref: https://www.kaggle.com/code/fchollet/keras-starter-kit-unet-train-on-full-dataset

On how the patch is draw / how the volume is create by concat / Where the validation set is

The notebook is the same setting with it.


In [9]:
import numpy as np
import torch.utils.data as data
import os
import PIL.Image as Image
from tqdm import tqdm
import glob
import torch.nn as nn
from torch import optim
import torch
from torch.utils.tensorboard import SummaryWriter



# ================== random patch dataset ==============================
is_local = (_dh != ["/kaggle/working"])

class RandomOpt():
    def __init__(self):
        self.SHARED_HEIGHT = 4096  # Height to resize all papyrii
        self.BUFFER = 64  # Half-size of papyrus patches we'll use as model inputs
        self.Z_DIM = 16  # Number of slices in the z direction. Max value is 64 - Z_START
        self.Z_START = 25  # Offset of slices in the z direction
        if is_local:
            self.DATA_DIR = "../data/raw/"
        else:
            self.DATA_DIR = "/kaggle/input/vesuvius-challenge-ink-detection"

def resize(img, SHARED_HEIGHT=RandomOpt().SHARED_HEIGHT):
    current_width, current_height = img.size
    aspect_ratio = current_width / current_height
    new_width = int(SHARED_HEIGHT * aspect_ratio)
    new_size = (new_width, SHARED_HEIGHT)
    img = img.resize(new_size)
    return img


def load_mask(split, index, DATA_DIR=RandomOpt().DATA_DIR):
    img = Image.open(f"{DATA_DIR}/{split}/{index}/mask.png").convert('1')
    img = resize(img)
    return torch.from_numpy(np.array(img))

def load_labels(split, index, DATA_DIR=RandomOpt().DATA_DIR):
    img = Image.open(f"{DATA_DIR}/{split}/{index}/inklabels.png")
    img = resize(img)
    return torch.from_numpy(np.array(img)).gt(0).float()

def load_volume(split, index, DATA_DIR=RandomOpt().DATA_DIR, Z_START=RandomOpt().Z_START, Z_DIM=RandomOpt().Z_DIM):
    # Load the 3d x-ray scan, one slice at a time
    z_slices_fnames = sorted(glob.glob(f"{DATA_DIR}/{split}/{index}/surface_volume/*.tif"))[Z_START:Z_START + Z_DIM]
    z_slices = []
    for z, filename in  tqdm(enumerate(z_slices_fnames)):
        img = Image.open(filename)
        img = resize(img)
        z_slice = np.array(img, dtype="float32")
        z_slices.append(torch.from_numpy(z_slice))
    return torch.stack(z_slices, dim=0)

# Random choice of patches for training
def sample_random_location(shape, BUFFER=RandomOpt().BUFFER):
    a=BUFFER
    random_train_x = (shape[0] - BUFFER - 1 - a)*torch.rand(1)+a
    random_train_y = (shape[1] - BUFFER - 1 - a)*torch.rand(1)+a
    random_train_location = torch.stack([random_train_x, random_train_y])
    return random_train_location

def is_in_masked_zone(location, mask):
    return mask[location[0].long(), location[1].long()]

def is_in_val_zone(location, val_location, val_zone_size, BUFFER=RandomOpt().BUFFER):
    x = location[0]
    y = location[1]
    x_match = val_location[0] - BUFFER <= x <= val_location[0] + val_zone_size[0] + BUFFER
    y_match = val_location[1] - BUFFER <= y <= val_location[1] + val_zone_size[1] + BUFFER
    return x_match and y_match

class RandomPatchLocDataset(data.Dataset):
    def __init__(self, mask, val_location, val_zone_size):
        self.mask = mask
        self.val_location = val_location
        self.val_zone_size = val_zone_size
        self.sample_random_location_train = lambda x: sample_random_location(mask.shape)
        self.is_in_mask_train = lambda x: is_in_masked_zone(x, mask)

    def is_proper_train_location(self, location):
        return not is_in_val_zone(location, self.val_location, self.val_zone_size) and self.is_in_mask_train(location)

    def __len__(self):
        return 1280

    def __getitem__(self, index):
        # Generate a random patch
        # Ignore the index
        loc = self.sample_random_location_train(0)
        while not self.is_proper_train_location(loc):
            loc = self.sample_random_location_train(0)
        return loc.int().squeeze(1)

## Define training and validation model
The full code with the model with Unet and random patch.

We will save the best model on validation.

NOTE: You can also download the model in OUTPUT and run test Code.

In [10]:
# ============= Model ==============
class ModelOpt:
    def __init__(self):
        # self.GPU_ID = '0'  
        self.Z_DIM = RandomOpt().Z_DIM
        self.BUFFER = RandomOpt().BUFFER
        self.SEED = 0
        self.BATCH_SIZE = 64
        self.LEARNING_RATE =1e-4
        self.TRAINING_EPOCH = 20
        self.LOG_DIR = '/kaggle/working'
        self.LOAD_VOLUME = [1, 2, 3]
        # Val
        self.VAL_LOC = (1300, 1000)
        self.VAL_SIZE = (300, 7000)

class RandomPatchModel():
    def __init__(self, opt = ModelOpt()):
        self.opt = opt
        self._setup_all()
        self.volume_list = [load_volume('train', i) for i in opt.LOAD_VOLUME]
        # Here volume: [Z_DIM, SHARED_HEIGHT, W_V1 + W_V2 + ...]
        self.volume = torch.cat(self.volume_list, dim=2)
        # Same for mask and label
        self.mask_list = [load_mask('train', i) for i in opt.LOAD_VOLUME]
        self.labels_list = [load_labels('train', i) for i in opt.LOAD_VOLUME]
        # [SHARED_HEIGHT, W_V1 + W_V2 + ...]
        self.labels = torch.cat(self.labels_list, dim=1)
        self.mask = torch.cat(self.mask_list, dim=1)

        self.net = UNet(in_ch=opt.Z_DIM).to(self.device)

        # Dataset
        self.loc_datast = RandomPatchLocDataset(self.mask, val_location=opt.VAL_LOC, val_zone_size=opt.VAL_SIZE)
        self.loc_loader = data.DataLoader(self.loc_datast, batch_size=opt.BATCH_SIZE)
        # Val
        self.val_loc = []
        for x in range(opt.VAL_LOC[0], opt.VAL_LOC[0] + opt.VAL_SIZE[0], opt.BUFFER):
            for y in range(opt.VAL_LOC[1], opt.VAL_LOC[1] + opt.VAL_SIZE[1], opt.BUFFER):
                if is_in_masked_zone([torch.tensor(x),torch.tensor(y)], self.mask):
                    self.val_loc.append([[x, y]])
        print(f"======> Num Patches Val: {len(self.val_loc)}")


    def _setup_all(self):
        # random seed
        np.random.seed(self.opt.SEED)
        torch.manual_seed(self.opt.SEED)
        torch.cuda.manual_seed_all(self.opt.SEED)
        # torch
        # os.environ['CUDA_VISIBLE_DEVICES'] = self.opt.GPU_ID
        torch.backends.cudnn.enabled = True
        torch.backends.cudnn.benchmark = True
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Log
        self.log_dir = self.opt.LOG_DIR
        self.ckpt = os.path.join(self.log_dir)

    def get_subvolume(self, batch_loc, volume, labels):
        # batch_loc : [batch_size, 2]
        subvolume = []
        label = []
        for l in batch_loc:
            x = l[0]
            y = l[1]
            sv = volume[:, x - self.opt.BUFFER:x + self.opt.BUFFER, y - self.opt.BUFFER:y + self.opt.BUFFER]
            sv = sv / 65535.
            subvolume.append(sv)
            if labels is not None:
                lb = labels[x - self.opt.BUFFER:x + self.opt.BUFFER, y - self.opt.BUFFER:y + self.opt.BUFFER]
                lb = lb.unsqueeze(0)
                label.append(lb)
        # [batch, Z_DIM, BUFFER, BUFFER]
        subvolume = torch.stack(subvolume)
        # [batch, 1, BUFFER, BUFFER]
        if labels is not None:
            label = torch.stack(label)
        return subvolume, label

    def augment_train_data(self, subvolume, label):
        # Add Data augmentation here
        return subvolume, label

    def train_loop(self):
        print("=====> Begin training")
        self.criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
        self.optimizer = optim.Adam(self.net.parameters(), lr=self.opt.LEARNING_RATE)
        self.net.train()

        best_val_loss = 100
        best_val_acc = 0
        meter = AverageMeter()
        for epoch in range(self.opt.TRAINING_EPOCH):
            bar = tqdm(enumerate(self.loc_loader), total=len(self.loc_datast) / self.opt.BATCH_SIZE)
            bar.set_description_str(f"Epoch: {epoch}")
            for i, loc in bar:
                subvolume, label = self.get_subvolume(loc, self.volume, self.labels)
                loss = self._train_step(subvolume, label)
                meter.update(loss)
                bar.set_postfix_str(f"Avg loss: {np.round(meter.get_value(),3)}")

            val_loss, val_acc = self.validataion_loop()
            print(f"======> Val Loss:{np.round(val_loss,3)} | Val Acc:{np.round(val_acc,3)} ")
            if val_loss < best_val_loss and val_acc > best_val_acc:
                torch.save(self.net.state_dict(), os.path.join(self.ckpt, "best.pt"))
                print("======> Save best val model")

                best_val_loss = val_loss
                best_val_acc = val_acc



    def _train_step(self, subvolume, label):
        self.optimizer.zero_grad()
        # inputs: subvolume: [batch, Z_DIM, BUFFER, BUFFER]
        #         label: [batch, 1, BUFFER, BUFFER]
        outputs = self.net(subvolume.to(self.device))
        loss = self.criterion(outputs, label.to(self.device))
        loss.backward()
        self.optimizer.step()
        return loss

    def validataion_loop(self):
        meter_loss = AverageMeter()
        meter_acc = AverageMeter()
        self.net.eval()
        for loc in self.val_loc:
            subvolume, label = self.get_subvolume(loc, self.volume, self.labels)
            outputs = self.net(subvolume.to(self.device))
            loss = self.criterion(outputs, label.to(self.device))
            meter_loss.update(loss)
            pred = torch.sigmoid(outputs) > 0.5
            meter_acc.update(
                (pred == label.to(self.device)).sum(),
                int(torch.prod(torch.tensor(label.shape)))
            )
        self.net.train()
        return meter_loss.get_value(), meter_acc.get_value()

    def load_best_ckpt(self):
        self.net.load_state_dict(torch.load(os.path.join(self.ckpt, "best.pt")))


# For the metric
class AverageMeter(object):
    def __init__(self):
        self.sum = 0
        self.n = 0

    def update(self, x, n=1):
        self.sum += float(x)
        self.n += n

    def reset(self):
        self.sum = 0
        self.n = 0

    def get_value(self):
        if self.n:
            return self.sum / self.n
        return 0

In [11]:
# Define model
model = RandomPatchModel()

16it [00:09,  1.76it/s]
0it [00:00, ?it/s]c:\Users\lzhen\miniconda3\envs\Vesuvius\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (140973980 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
16it [00:09,  1.60it/s]
16it [00:04,  3.41it/s]


======> Num Patches Val: 347


In [12]:
# Training
model.train_loop()

=====> Begin training


Epoch: 0: 100%|██████████| 20/20.0 [00:26<00:00,  1.31s/it, Avg loss: 0.587]


======> Val Loss:0.594 | Val Acc:0.814 


RuntimeError: Parent directory /kaggle does not exist.

## Testing

In [ ]:
import torch

# Load the best model
model.load_best_ckpt()
model.criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
loss, acc = model.validataion_loop()
model.net.eval()
print(f"Val loss: {np.round(loss,3)} | Val acc: {np.round(acc, 3)}")

In [ ]:
from tqdm import tqdm
def compute_predictions_map(split, index):
    print(f"======> Load data for {split}/{index}")
    test_volume = load_volume(split=split, index=index)
    test_mask = load_mask(split=split, index=index)
    print(f"======> Volume shape: {test_volume.shape}")
    test_locations = []
    BUFFER = model.opt.BUFFER
    stride = BUFFER // 2

    for x in range(BUFFER, test_volume.shape[1] - BUFFER, stride):
        for y in range(BUFFER, test_volume.shape[2] - BUFFER, stride):
            if is_in_masked_zone([torch.tensor(x),torch.tensor(y)], test_mask):
                test_locations.append((x, y))
    print(f"======> {len(test_locations)} test locations (after filtering by mask)")

    predictions_map = torch.zeros((1, 1, test_volume.shape[1], test_volume.shape[2]))
    predictions_map_counts = torch.zeros((1, 1, test_volume.shape[1], test_volume.shape[2]))
    print(f"======> Compute predictions")

    with torch.no_grad():
        bar = tqdm(test_locations)
        for loc in bar:
            subvolume, label = model.get_subvolume([loc], test_volume, None)
            outputs = model.net(subvolume.to(model.device))
            pred = torch.sigmoid(outputs)
            # print(loc, (pred > 0.5).sum())
            # Here a single location may be with multiple result
            predictions_map[:, :, loc[0] - BUFFER : loc[0] + BUFFER, loc[1] - BUFFER : loc[1] + BUFFER] += pred.cpu()
            predictions_map_counts[:, :, loc[0] - BUFFER : loc[0] + BUFFER, loc[1] - BUFFER : loc[1] + BUFFER] += 1

    # print(predictions_map_b[:,:, 2500, 1000])
    # print(predictions_map_counts[:,:, 2500, 1000])
    predictions_map /= (predictions_map_counts + 1e-7)
    return predictions_map

In [ ]:
predictions_map_a = compute_predictions_map(split="test", index="a")
predictions_map_b = compute_predictions_map(split="test", index="b")

In [ ]:
import matplotlib.pyplot as plt
# Threshold is very important !!!!!
plt.imshow(predictions_map_a.squeeze() > 0.10, cmap='gray')

In [ ]:
plt.imshow(predictions_map_b.squeeze() > 0.10, cmap='gray')

## Submission

Rescale the pred to the raw size and create 'submission.csv'

In [ ]:
from skimage.transform import resize as resize_ski
import PIL.Image as Image

DATA_DIR = "/kaggle/input/vesuvius-challenge-ink-detection"
original_size_a = Image.open(DATA_DIR + "/test/a/mask.png").size
original_size_b = Image.open(DATA_DIR + "/test/b/mask.png").size
predictions_map_a = resize_ski(predictions_map_a.squeeze(), original_size_a).squeeze()
predictions_map_b = resize_ski(predictions_map_b.squeeze(), original_size_b).squeeze()

In [ ]:
def rle(predictions_map, threshold):
    flat_img = predictions_map.flatten()
    flat_img = np.where(flat_img > threshold, 1, 0).astype(np.uint8)

    starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
    ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
    starts_ix = np.where(starts)[0] + 2
    ends_ix = np.where(ends)[0] + 2
    lengths = ends_ix - starts_ix
    return " ".join(map(str, sum(zip(starts_ix, lengths), ())))

threshold_a = 0.10
threshold_b = 0.10

rle_a = rle(predictions_map_a, threshold=threshold_a)
rle_b = rle(predictions_map_b, threshold=threshold_b)
print("Id,Predicted\na," + rle_a + "\nb," + rle_b, file=open('/kaggle/working/submission.csv', 'w'))

## Next step
You can
- Try different architecture in Unet:
    - Other size
    - Attention Block
- Tune the hyperparameter
    - Z_DIM ...
- Use more suitable segmentation Loss

In [ ]:
# END